# Loadning required libraries:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
from pandas.plotting import scatter_matrix
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('always')

# Loading the Dataset:

In [ ]:
df = pd.read_csv("air.csv")
pd.set_option('display.max_columns', None)#to view the all 45 columns.
df.head()

# Column Name Description:

FL_DATE: Date of the flight, mm/dd/yyyy

OP_CARRIER: Airline Identifier

OP_CARRIER_FL_NUM: Flight Number

ORIGIN: Starting Airport Code

DEST: Destination Airport Code

CRS_DEP_TIME: Planned Departure Time

DEP_TIME: Actual Departure Time

DEP_DELAY: Total Delay on Departure in minutes

TAXI_OUT: The time duration elapsed between departure from the origin airport gate and wheels off

WHEELS_OFF: The time point that the aircraft's wheels leave the ground

WHEELS_ON: The time point that the aircraft's wheels touch on the ground

TAXI_IN: The time duration elapsed between wheels-on and gate arrival at the destination airport

CRS_ARR_TIME: Planned arrival time

ARR_TIME: Actual Arrival Time

ARR_DELAY: Total Delay on Arrival in minutes

CANCELLED: Flight Cancelled (1 = cancelled)

CANCELLATION_CODE: Reason for Cancellation of flight: A - Airline/Carrier; B - Weather; C - National                    Air System; D - Security

DIVERTED: Aircraft landed on airport that out of schedule

CRS_ELAPSED_TIM: Planned time amount needed for the flight trip

ACTUAL_ELAPSED_TIME: AIR_TIME+TAXI_IN+TAXI_OUT

AIR_TIME: The time duration between wheels_off and wheels_on time

DISTANCE: Distance between two airports

CARRIER_DELAY: Delay caused by the airline in minutes

WEATHER_DELAY: Delay caused by weather

NAS_DELAY: Delay caused by air system

SECURITY_DELAY: Delay caused by security

LATE_AIRCRAFT_DELAY: Delay caused by security

Unnamed: 44 - Useless column(100% NULL Values)

In [ ]:
df.shape# we have totally 448620 row/observations and 45 columns/variables

In [ ]:
df.info()#to check the data types and null values if any.

In [ ]:
df.isnull().sum()#to check the total missing values in each columns.

In [ ]:
df.describe()#to summarize the dataset.

### Exploring Dataset:

In [ ]:
df.YEAR.unique()# Let's take the first column "YEAR" it has only 1987 for all 448620 rows.

In [ ]:
df.MONTH.unique()#In second column only from month of Octomber.

In [ ]:
df.DAY_OF_MONTH.unique()# in third column shows all day of october month.

In [ ]:
df.DAY_OF_WEEK.unique()#in fourth column 1-Mon, 2-Tue, 3-Wed, 4-Thu, 5-Fri, 6-Sat, 7-Sun.

In [ ]:
df.FL_DATE.unique()#in fifth column it has only the date of Oct-1987.

# OBSERVATION: 01

 As per the above observations we come to know that our dataset has a details of airways for the month of Octomber in 1987.
 
 Since we have the "FL_DATE" column we can remove the first 4 columns as "YEAR", "MONTH", "DAY_OF_MONTH", "DAY_OF_WEEK" from our dataset "df".

In [ ]:
#Lets remove the 4 columns

df.drop(['YEAR','MONTH','DAY_OF_MONTH','DAY_OF_WEEK'], axis = 1,inplace = True)
df.head()

In [ ]:
df.shape # after removed the 4 columns we reduced our columns from 45 to 41. 

In [ ]:
df.columns # after FL_DATE we can explore UNIQUE_CARRIER.

In [ ]:
airlines=df.UNIQUE_CARRIER.unique() #Airline codes

In [ ]:
len(airlines) #so totally we have 14 airlines details in the month of Octomber.

In [ ]:
airlines# airlines are code are gathered from wiki.

#### Airline Abbreviation:

AA - American Airlines

US - Unisted State Airways

AS - Alska Airlines

CO - Continental Airlines

DL - Delta Airlines

EA - Easter Airlines

HP - Hawaiian Pacific Airlines

NW - Northwest Airlines

PA - Pan American Airways

PI - Piedmont Airlines

PS - Ukraine International

TW - Twan Air

UA - United Airlines

WN - Southwest Airlines

In [ ]:
#after UNIQUE_CARRIER column we can see a empty column name "TAIL_NUM" so we remove it since its 
#totally empty.

df.drop(['TAIL_NUM'], axis = 1, inplace = True)

In [ ]:
df.head()

In [ ]:
#now our third column is FL_NUM:

df.FL_NUM.unique()

In [ ]:
fl=df.groupby('FL_NUM')

In [ ]:
fl.first()

In [ ]:
# Since there is no any relationship between anyothere columns lets drop "FL_NUM".

df.drop(['FL_NUM'],axis=1)

In [ ]:
df.columns[df.isnull().any()] # to find the column names for the missing values.

# Handling Missing Values:

In [ ]:
#To calculate the percentage of the missing values:

missing_df = df.isnull().sum(axis=0).reset_index()
missing_df.columns = ['variable', 'missing values']
missing_df['filling factor (%)']=(df.shape[0]-missing_df['missing values'])/df.shape[0]*100
missing_df.sort_values('filling factor (%)').reset_index(drop = True)

In [ ]:
df.shape

# OBSERVATION: 02

As per above, Now we can see that the missing values are >99% since its not gonna affect our model at all. However we can also see that 12 columns are totally 0% i.e., it is totally zero so lets remove the 12 columns.

In [ ]:
df.drop(['Unnamed: 44','WHEELS_ON','CANCELLATION_CODE','TAXI_IN','LATE_AIRCRAFT_DELAY','TAXI_OUT','WHEELS_OFF','AIR_TIME','CARRIER_DELAY','WEATHER_DELAY','NAS_DELAY','SECURITY_DELAY'], axis = 1, inplace = True)

In [ ]:
df.shape# now our dataset has only 28 variables from 45.

In [ ]:
corr = df.corr()
corr.style.background_gradient()

# OBSERVATION: 03

 As you can see in the column nameS "FLIGHTS" and "CANCELLED" its shows mostly it has some nan value so lets explore on it.

In [ ]:
df.FLIGHTS.unique()#for all 448620 it has onle "1" so it doesn't mean anything lets drop it.

In [ ]:
df.drop(['FLIGHTS'],axis = 1, inplace=True)

In [ ]:
df.shape#now we reduced the variable from 28 to 27.

In [ ]:
df.CANCELLED.unique()#1-cancelled, 0-non_cancelled.

In [ ]:
can=df[(df['CANCELLED'])>0]

In [ ]:
len(can)#total cancelled flights we have 3001.

In [ ]:
pd.set_option('display.max_rows', None)
can

# OBSERVATION:04

 1.Since the flight was cancelled(Not opereated) under column names DEP_TIME, DEP_DELAY, DEP_DELAY_NEW, DEP_DEL15, DEP_DELAY_GROUP, ARR_TIME	ARR_DELAY	ARR_DELAY_NEW	ARR_DEL15	ARR_DELAY_GROUP and ACTUAL_ELAPSED_TIME	are obviously we will get "NAN" values.
 
 2.As you can see the  column name DEP_DELAY_NEW, DEP_DELAY_GROUP, ARR_DELAY_NEW, ARR_DELAY_GROUP are not necessary so we can drop it.

In [ ]:
df.columns

In [ ]:
df.drop(['ARR_DELAY_NEW','ARR_DELAY_GROUP','DEP_DELAY_NEW','DEP_DELAY_GROUP','DISTANCE_GROUP'],axis=1, inplace = True)

In [ ]:
df.shape#now again we reduced the variable from 27 to 22.

In [ ]:
corr = df.corr()
corr.style.background_gradient()

# OBSERVATION:05

 Now we can see some clear correlations in our dataset.

# ARRIVAL DELAYS BY DATE:

In [ ]:
df.groupby('FL_DATE').ARR_DELAY.sum().plot.bar()
plt.title('ARRIVAL DELAYS BY DATE')
plt.ylabel('Hours')
plt.xlabel('Month of the year')

# DEPARTURE DELAYS BY DATE:

In [ ]:
df.groupby('FL_DATE').DEP_DELAY.sum().plot.bar()
plt.title('DEPARTURE DELAYS BY DATE')
plt.ylabel('Hours')
plt.xlabel('Date of Departure')

# DEPARTURE DELAYS BY > 15:

In [ ]:
df.groupby('FL_DATE').DEP_DEL15.sum().plot.bar()
plt.title('DEPARTURE DELAYS BY >15')
plt.ylabel('Hours')
plt.xlabel('Date of Departure')

# ARRIVAL DELAYS BY > 15:

In [ ]:
df.groupby('FL_DATE').ARR_DEL15.sum().plot.bar()
plt.title('ARRIVAL DELAYS BY >15')
plt.ylabel('Hours')
plt.xlabel('Date of Departure')

# DELAYS BY AIRLINES:

In [ ]:
df.groupby('UNIQUE_CARRIER').ARR_DELAY.sum().sort_values(ascending=False).plot.bar()
plt.title('AIRWAYS DELAYS IN OCTOBER')
plt.ylabel('Hours')
plt.xlabel('Date of Departure')

# DEPARTURE DELAYS BY CITIES:

In [ ]:
df.groupby('ORIGIN').DEP_DELAY.sum().sort_values(ascending=False).plot.bar()
plt.title('DEPARTURE DELAY BY CITIES')

##### As per above plot we can get the clear picture so let's grab only top 20.

In [ ]:
city_dep_delay = df.groupby('ORIGIN').DEP_DELAY.sum().sort_values(ascending=False)
city_dep_delay[:20].plot.bar()
plt.title('DEPARTURE DELAY BY TOP 20 CITIES')

# OBSERVATION: 06

 As we can see in the above plots for the delay in Departure/Arrival mostly the maximum delays happened in the weekends.

10-01-1987 - Thursday

10-09-1987 - Friday

10-16-1987 - Friday

10-22-1987 - Thursday

10-23-1987 - Friday

10-25-1987 - Sunday

10-28-1987 - Wednesday

10-29-1987 - Thursday

# OBSERVATIONS: 07

 As we can see in the above plots for the delay by airways mostly the maximum delays happened in the DL - Delta Airlines and the least delays in Pan America Airways. 

##### To check the maximum delay by the date: 

In [ ]:
df.groupby('FL_DATE')[['ARR_DELAY']].max().sort_values(by='ARR_DELAY',ascending=False)

##### To check the total number of airport :

In [ ]:
print("No of airports: {}".format(len(df['ORIGIN'].unique())))

##### To check the top 10 cities by departures:

In [ ]:
plt.figure(figsize=(10, 10))
axis = sns.countplot(x=df['ORIGIN'], data = df,order=df['ORIGIN'].value_counts().iloc[:10].index)
axis.set_xticklabels(axis.get_xticklabels(), rotation=90, ha="right")
plt.tight_layout()
plt.show()

##### To check the top 10 airways:

In [ ]:
plt.figure(figsize=(10, 10))
axis = sns.countplot(x=df['UNIQUE_CARRIER'], data = df,order=df['UNIQUE_CARRIER'].value_counts().iloc[:10].index)
axis.set_xticklabels(axis.get_xticklabels(), rotation=90, ha="right")
plt.tight_layout()
plt.show()

##### Let's drop the NA's in all rows to see the correlations:

In [ ]:
df1 = df.dropna()
print("Shape of old dataset:",df.shape)
print("Shape of non-null dataset:",df1.shape)

# CORRELATION:

In [ ]:
axis = plt.subplots(figsize=(20,14))
sns.heatmap(df1.corr(),annot = True)
plt.show()

# OBSERVATIONS: 08

 1. As we can there is high correlation between ARR_TIME and DEP_TIME, ACTUAL_ELAPSED_TIME and DISTANCE.
 2. You can see the "CANCELLED" and "DIVERTED" columns are blanks since it has only "0" and "1" and also we don't need this column to predict our delayed flights.

##### Dropping "CANCELLED" and "DIVERTED" columns:

In [ ]:
df2=df1.drop(["CANCELLED","DIVERTED"],axis=1,inplace = True)

In [ ]:
df.shape

In [ ]:
df1.shape

In [ ]:
df1.isnull().sum()# now there is no any missing values lets begin to build our model.

# MODEL BUILDING:

In [ ]:
df1.dtypes

In [ ]:
#to change categorical variables into numerical one

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df1['FL_DATE']= le.fit_transform(df1['FL_DATE'])
df1['UNIQUE_CARRIER']= le.fit_transform(df1['UNIQUE_CARRIER'])
df1['ORIGIN']= le.fit_transform(df1['ORIGIN'])
df1['ORIGIN_STATE_ABR']= le.fit_transform(df1['ORIGIN_STATE_ABR'])
df1['DEST']= le.fit_transform(df1['DEST'])
df1['DEST_STATE_ABR']= le.fit_transform(df1['DEST_STATE_ABR'])

In [ ]:
df1.dtypes

Let's split 'x' and 'y' from our dataset 'df1'.

In [ ]:
y=df1['ARR_DELAY']#dependent variavle "Y"
y.head(2)

In [ ]:
x=df1.drop('ARR_DELAY',axis = 1)
x.head(2)

In [ ]:
# Splitting into train and test dataset:

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state = 40)

##### Lets check the shape of the splitted datasets:

In [ ]:
print("x_train shape is:",x_train.shape)
print("x_test shape is:",x_test.shape)
print("y_train shape is:",y_train.shape)
print("y_test shape is:",y_test.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc1=StandardScaler()
x_train_sc=sc1.fit_transform(x_train)
x_test_sc=sc1.transform(x_test)

In [ ]:
from sklearn.linear_model import Lasso,LinearRegression,Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

Las = Lasso()
LinR = LinearRegression()
Rid = Ridge()
Rfc = RandomForestRegressor(random_state=2)
Dtc = DecisionTreeRegressor(random_state = 2)

for model, name in zip([Las,LinR,Rid,Dtc,Rfc],['Lasso','Linear Regression','Ridge','Random forest Regressor','Decision Tree Regressor']):
    model1 = model.fit(x_train_sc,y_train)
    Y_predict=model1.predict(x_test_sc)
    print(name)
    print('Mean Absolute Error:', mean_absolute_error(y_test, Y_predict))  
    print('Mean Squared Error:', mean_squared_error(y_test, Y_predict))  
    print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, Y_predict)))
    print('R2 : ',r2_score(y_test, Y_predict))
    print()

# Visualizing the models:

In [ ]:
for model, name in zip([Las,LinR,Rid,Dtc,Rfc], ['Lasso','Linear Regression','Ridge','Random forest Regressor','Decision Tree Regressor']):
    model1 = model.fit(x_train_sc,y_train)
    Y_predict=model1.predict(x_test_sc)
    print(name)
    plt.scatter(y_test, Y_predict)
    plt.title("Model Analysis")
    plt.xlabel("Truth")
    plt.ylabel("Prediction")
    plt.show()

In [ ]:
pd.DataFrame({'Actual': y_test, 'Predicted': Y_predict}).head(10)